### 注意
股票收盤是 13:30，如果直接查詢當日，可能資料還沒出來

## 偽代碼
查詢台灣證券交易所全部資料，返回 json 格式來進行解析
> 嘗試直接讀取網頁，但是回來的 html 格式解析遠不及 json 方便！

- 使用 datetime.today() 取得今天日期
- 如果已經存在查詢日期的 csv, 就不再進行請求
- 使用 requests 對 http://www.twse.com.tw/exchangeReport/MI_INDEX 做 GET 請求
- 夾帶 query string data 有
    response: json
    date: 20190402 => 使用 datetime.today 取得今天，再用組合的方式得到這個格式
    type: ALL
    _: 1554259760082 時戳 str(round(time.time() * 1000) - 500)
    
- 返回成功的話就進行解析，不成功就 return 發錯誤訊息 '今天還沒收盤'
- 整理資料並且存成 csv 

In [1]:
import requests
import pandas as pd
import datetime, time

In [8]:
today = datetime.datetime.today()
print('{0}{:2}{:2}'.format(today.year, today.month, today.day))

ValueError: cannot switch from manual field specification to automatic field numbering

In [11]:
today = '20190403'

In [12]:
url = 'http://www.twse.com.tw/exchangeReport/MI_INDEX'
params = {
    'response': 'json',
    'date': today,
    'type': 'ALL',
    '_': str(round(time.time() * 1000) - 500)
}

result = requests.get(url, params=params)
content = result.json()

In [13]:
content.keys()

dict_keys(['data4', 'fields1', 'subtitle4', 'notes4', 'data5', 'date', 'alignsStyle1', 'groups5', 'fields4', 'stat', 'alignsStyle2', 'data3', 'data1', 'fields3', 'subtitle5', 'fields2', 'alignsStyle4', 'subtitle1', 'fields5', 'alignsStyle5', 'data2', 'params', 'alignsStyle3', 'notes5'])

In [21]:
# 報酬指數 不是我要的資料
# content['data2']

In [22]:
# 股價指數 不是我要的資料
# content['data1']

In [24]:
# 不是要找的資料
# content['data3']

In [26]:
# content['data4']

In [30]:
'red' in df['漲跌(+/-)'][0]

True

In [15]:
# Bingo!
# 進行資料整理，先給欄位
columns = ['證券代號', '證券名稱', '成交股數', '成交筆數', '成交金額', '開盤價', '最高價', '最低價', '收盤價', '漲跌(+/-)', '漲跌價差', '最後揭示買價', '最後揭示買量', '最後揭示賣價', '最後揭示賣量', '本益比']
stock = content['data5']
df = pd.DataFrame(stock, columns=columns)

In [18]:
def format_漲跌(val):
    if 'red' in val:
        return '+'
    if 'green' in val:
        return '-'

    return ''

df['漲跌(+/-)'] = df['漲跌(+/-)'].apply(format_漲跌)

In [19]:
df

,證券代號,證券名稱,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌(+/-),漲跌價差,最後揭示買價,最後揭示買量,最後揭示賣價,最後揭示賣量,本益比
0,0050,元大台灣50,"6,867,174","3,017","550,702,889",80.40,80.45,79.95,80.15,,0.00,80.10,11,80.15,251,0.00
1,0051,元大中型100,"24,400",19,"781,694",32.06,32.07,31.93,32.04,+,0.09,32.04,19,32.06,3,0.00
2,0052,富邦科技,"104,102",12,"5,656,997",54.00,54.35,54.00,54.35,+,0.40,51.65,3,54.40,1,0.00
3,0053,元大電子,"11,000",5,"382,470",34.73,34.85,34.73,34.85,+,0.03,34.70,1,34.88,17,0.00
4,0054,元大台商50,"18,418",17,"413,512",22.35,22.49,22.35,22.49,+,0.03,22.38,1,22.48,1,0.00
5,0055,元大MSCI金融,"16,672",11,"290,262",17.40,17.42,17.40,17.41,+,0.01,17.41,12,17.42,28,0.00
6,0056,元大高股息,"10,981,802","3,369","293,488,073",26.62,26.76,26.62,26.73,+,0.15,26.73,73,26.74,59,0.00
7,0057,富邦摩台,"26,000",9,"1,314,900",50.55,50.80,50.30,50.60,+,0.05,50.20,20,50.70,25,0.00
8,0058,富邦發達,"25,000",5,"1,154,820",46.06,46.25,45.94,46.25,-,0.07,45.87,1,46.39,1,0.00
9,0059,富邦金融,"2,000",2,"87,900",43.95,43.95,43.95,43.95,+,0.28,43.67,1,--,0,0.00


In [20]:
df.to_csv('{0}.csv'.format(today))

In [22]:
# 取資料證券代號從 1101 開始，且只有4位數
df2 = df[df['證券代號'].str.len() == 4]

,證券代號,證券名稱,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌(+/-),漲跌價差,最後揭示買價,最後揭示買量,最後揭示賣價,最後揭示賣量,本益比
0,0050,元大台灣50,"6,867,174","3,017","550,702,889",80.40,80.45,79.95,80.15,,0.00,80.10,11,80.15,251,0.00
1,0051,元大中型100,"24,400",19,"781,694",32.06,32.07,31.93,32.04,+,0.09,32.04,19,32.06,3,0.00
2,0052,富邦科技,"104,102",12,"5,656,997",54.00,54.35,54.00,54.35,+,0.40,51.65,3,54.40,1,0.00
3,0053,元大電子,"11,000",5,"382,470",34.73,34.85,34.73,34.85,+,0.03,34.70,1,34.88,17,0.00
4,0054,元大台商50,"18,418",17,"413,512",22.35,22.49,22.35,22.49,+,0.03,22.38,1,22.48,1,0.00


In [26]:
df2 = df2.loc[13680:]

In [29]:
columns = ['最後揭示買價', '最後揭示買量', '最後揭示賣價', '最後揭示賣量']
df2.drop(columns, inplace=True, axis=1)

/Users/ray/anaconda3/envs/generic/lib/python3.7/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [30]:
df2.head()

,證券代號,證券名稱,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌(+/-),漲跌價差,本益比
13680,1101,台泥,"19,088,018","5,804","794,119,481",41.55,41.75,41.50,41.70,+,0.30,10.45
13682,1102,亞泥,"6,428,716","3,514","265,058,031",41.50,41.50,40.85,41.30,+,0.05,12.48
13683,1103,嘉泥,"1,256,926",365,"18,757,528",14.95,15.05,14.80,15.05,+,0.15,13.81
13684,1104,環泥,"203,265",132,"4,023,843",19.85,19.85,19.75,19.85,+,0.05,12.25
13685,1108,幸福,"121,760",50,"897,381",7.46,7.46,7.28,7.32,-,0.14,0.00


## 策略

1. 昨天的收盤價 > 昨天的開盤價（昨天收黑）
 - 昨天的最高價 > 今天的最高價
 - 昨天的最低價 < 今天的最低價


2. 前天的收盤價 > 前天的開盤價（前天收黑）
 - 前天的最高價 > 昨天的最高價
 - 前天的最低價 < 昨天的最低價
 
 
3. 今天的收盤價 and 最高價 > 前天的最高價

In [ ]:
def black():
    pass

def choose(previous, current):
    pass

def 